## Learn about class and pytorch

In [15]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Pytorch ingnite

In [16]:
!pip install pytorch-ignite

     |████████████████████████████████| 221 kB 6.9 MB/s 


In [17]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

# Build data loader

In [21]:
import pyarrow.feather as feather
import numpy as np
from sklearn.model_selection import train_test_split



In [35]:
fluxData_df = feather.read_feather('data/fluxData.feather')
zernikeData_df = feather.read_feather('data/zernikeData.feather')
X_train, X_val, y_train, y_val = train_test_split(fluxData_df, zernikeData_df, test_size=0.33, random_state=42)


train_target = torch.tensor(y_train.values.astype(np.float32))
trainInput = torch.tensor(X_train.values.astype(np.float32))

train_tensor = torch.utils.data.TensorDataset(trainInput, train_target) 
loaderTrain = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = 64, shuffle = True)


valid_target = torch.tensor(zernikeData_df.values.astype(np.float32))
validInput = torch.tensor(fluxData_df.values.astype(np.float32))

train_tensor = torch.utils.data.TensorDataset(validInput, valid_target) 
loaderValid = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = 3, shuffle = True)



In [36]:
next(iter(loaderValid))

[tensor([[3.9838, 0.7945, 1.1854, 0.5048, 1.2628, 0.7608, 1.8350, 1.7367, 0.3203,
          1.0304, 1.1907, 1.0754, 0.3224, 0.7151, 1.1101, 0.6950, 0.3608, 0.2311,
          0.2402],
         [3.6214, 1.4549, 1.7598, 1.0048, 1.4599, 0.6538, 0.9810, 1.5470, 0.2775,
          2.0602, 0.8395, 0.8128, 0.3921, 0.4869, 1.0023, 0.8948, 0.3750, 0.1900,
          0.2975],
         [4.8043, 1.1849, 1.0833, 0.8343, 1.1422, 0.8732, 2.6207, 0.6068, 0.2824,
          0.7544, 0.9943, 0.6023, 0.7088, 0.3697, 0.3085, 0.4367, 0.2326, 0.3506,
          0.2417]]),
 tensor([[-0.2079, -0.3518,  0.1227,  0.3991,  0.0929,  0.2048, -0.1164, -0.2364,
           0.0084],
         [-0.2178, -0.3519,  0.1679,  0.4260, -0.0993,  0.1191,  0.1388,  0.0766,
           0.0916],
         [ 0.0759, -0.1005, -0.1488, -0.0856,  0.2493, -0.2654, -0.1965,  0.2039,
           0.2004]])]

In [44]:
loaderValid.batch_sampler.sampler.num_samples

58740

This cell defines the four pieces of our training procedure:
`build_dataset`, `build_network`, `build_optimizer`, and `train_epoch`.

All of these are a standard part of a basic PyTorch pipeline,
and their implementation is unaffected by the use of W&B,
so we won't comment on them.

In [18]:
def build_dataset(batch_size):
    fluxData_df = feather.read_feather('data/fluxData.feather')
    zernikeData_df = feather.read_feather('data/zernikeData.feather')
    X_train, X_test, y_train, y_test = train_test_split(
...     fluxData_df, zernikeData_df, test_size=0.33, random_state=42)


    train_target = torch.tensor(zernikeData_df.values.astype(np.float32))
    train = torch.tensor(fluxData_df.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(train, train_target) 
    loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
   


    return loader

def build_network():
    network = nn.Sequential( 
    nn.Linear(19,2000), nn.ReLU(),
    nn.Linear(2000,1050), nn.ReLU(),
    nn.Linear(1050,100), nn.ReLU(),
    nn.Linear(100, 9))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
       # print(" hello train_epoch")


        # ➡ Forward pass
        loss = nn.MSELoss()
        loss =loss(network(data), target)
        #cumu_loss += loss
        lossRMS= nn.MSELoss()
        RmsLossValue=torch.sqrt(lossRMS(torch.flatten(network(data)), torch.flatten(target)))

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        #wandb.log({"batch loss": RmsLossValue, "RMS Loss" : RmsLossValue})

    return RmsLossValue

SyntaxError: invalid syntax (<ipython-input-18-638d2d89242b>, line 5)

In [5]:
train()

 hello train
0
 hello poch ende
tensor(0.0476, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
1
 hello poch ende
tensor(0.0427, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
2


KeyboardInterrupt: 

## Build AO network

In [12]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(19,2000)
        self.linear2 = nn.Linear(2000,1050)
        self.linear3 = nn.Linear(1050,100)
        self.linear3 = nn.Linear(1050,100)
        self.out = nn.Linear(100,19)
        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU()
    })

    def forward(self, x, act):
        x = self.linear1(x)
        x = self.activations[act](x)
        x = self.linear2(x)
        x = self.activations[act](x)
        x = self.linear3(x)
        x = self.activations[act](x)
        x = self.out(x)
        return(x)


ClassNetwork = Net()
sample_input = torch.randn(19)
ClassNetwork(sample_input, "relu")
#print(ClassNetwork)

tensor([ 0.0156, -0.0943,  0.0032,  0.1251, -0.1313,  0.0721, -0.0457,  0.1579,
         0.1255,  0.0058,  0.0656, -0.0503, -0.0087, -0.0416,  0.1287, -0.1405,
        -0.0949,  0.0222,  0.1019], grad_fn=<AddBackward0>)